In [ ]:
!git clone https://github.com/soumik12345/point-cloud-segmentation
!pip install -qqq wandb ml_collections keras_tuner

In [ ]:
import sys

sys.path.append("..")

In [ ]:
import os

from datetime import datetime

from tensorflow.keras import optimizers, callbacks
from tensorflow.keras import mixed_precision
from keras_tuner import RandomSearch
import wandb

from point_seg import TFRecordLoader, ShapeNetCoreLoaderInMemory
from point_seg import models, utils

In [ ]:
object_category = (
    "Airplane"  # @param ["Airplane", "Bag", "Car", "Chair"] {type:"string"}
)
num_points = 1024  # @param {type:"integer"}

In [ ]:
timestamp = datetime.utcnow().strftime("%y%m%d-%H%M%S")
strategy = utils.initialize_device()
batch_size = 32 * strategy.num_replicas_in_sync

In [ ]:
wandb.init(
    project="pointnet_shapenet_core",
    name=f"{object_category}_{timestamp}_tuning",
    entity="pointnet",
    sync_tensorboard=True,
)

In [ ]:
# Apply mixed-precision policy [OPTIONAL]
mixed_precision.set_global_policy("mixed_float16")
policy = mixed_precision.global_policy()

In [ ]:
data_loader = ShapeNetCoreLoaderInMemory(
    object_category=object_category, n_sampled_points=num_points,
)
data_loader.load_data()
train_dataset, val_dataset = data_loader.get_datasets(
    val_split=0.2, batch_size=batch_size,
)

In [ ]:
def sample_model(hp):
    with strategy.scope():
        _, y = next(iter(train_dataset))
        num_classes = y.shape[-1]
        model = models.get_shape_segmentation_model(num_points, num_classes)

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model


class CustomTuner(RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        initial_lr = trial.hyperparameters.Float(
            "learning_rate", 1e-4, 1e-2, sampling="log", default=1e-3
        )
        drop_every = trial.hyperparameters.Choice("drop_lr_epoch", values=[10, 15, 20])

        lr_scheduler = utils.StepDecay(initial_lr, drop_every, decay_factor=0.5)
        lr_callback = callbacks.LearningRateScheduler(
            lambda epoch: lr_scheduler(epoch), verbose=True
        )
        tb_callback = callbacks.TensorBoard(log_dir=wandb.run.dir)

        epochs = trial.hyperparameters.Int(
            "epochs", min_value=50, max_value=80, step=10
        )

        kwargs["epochs"] = epochs
        kwargs["callbacks"] = [lr_callback, tb_callback]
        super(CustomTuner, self).run_trial(trial, *args, **kwargs)

In [ ]:
tb_callback = callbacks.TensorBoard(log_dir=wandb.run.dir)

tuner = CustomTuner(
    sample_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory=f"{object_category}_{timestamp}",
    project_name="pointnet-segmentation",
)

tuner.search(train_dataset, validation_data=val_dataset)